# 1. MLP로 XOR 구현하기

In [4]:
import torch
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

#### 1.1 단층 퍼셉트론 시도

In [8]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

linear = nn.Linear(2,1)
sigmoid = nn.Sigmoid()
model = nn.Sequential(linear, sigmoid).to(device)

criterion = torch.nn.BCELoss().to(device)               # 이진 분류에 사용하는 Cross-Entropy 함수
optimizer = torch.optim.SGD(model.parameters(), lr=1)

for step in range(10001) :

  optimizer.zero_grad()
  hyp = model(X)
  cost = criterion(hyp, Y)
  cost.backward()
  optimizer.step()

  if step % 2000 == 0: 
      print(step, cost.item())  # 줄어들지 않는다. XOR은 단층 퍼셉트론으로는 풀 수 없다.

0 0.70228111743927
2000 0.6931471824645996
4000 0.6931471824645996
6000 0.6931471824645996
8000 0.6931471824645996
10000 0.6931471824645996


####1.2 다층 퍼셉트론 설계

In [11]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

model = nn.Sequential(
      nn.Linear(2, 10, bias=True),    # 입력
      nn.Sigmoid(),
      nn.Linear(10,10, bias=True),    # hidden 1
      nn.Sigmoid(),
      nn.Linear(10,10, bias=True),    # hidden 2
      nn.Sigmoid(),
      nn.Linear(10, 1, bias=True),    # 출력
      nn.Sigmoid()                    
).to(device)

criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

for epoch in range(10001) :

  hyp = model(X)
  cost = criterion(hyp, Y)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 2000 == 0:
      print(epoch, cost.item())

0 0.7006749510765076
2000 5.9339705330785364e-05
4000 1.3590030903287698e-05
6000 4.321354026615154e-06
8000 1.4901181657478446e-06
10000 5.662444095833052e-07


#### 1.3 학습한 MLP 성능 Test

In [13]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

with torch.no_grad() :
  hyp = model(X)
  pred = (hyp>0.5).float()
  accuracy = (pred==Y).float().mean()

  print('모델의 출력값(Hypothesis): ', hyp.detach().cpu().numpy())
  print('모델의 예측값(Predicted): ', pred.detach().cpu().numpy())
  print('실제값(Y): ', Y.cpu().numpy())
  print('정확도(Accuracy): ', accuracy.item())

모델의 출력값(Hypothesis):  [[3.8152713e-08]
 [9.9999893e-01]
 [9.9999893e-01]
 [3.8867977e-08]]
모델의 예측값(Predicted):  [[0.]
 [1.]
 [1.]
 [0.]]
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  1.0


## 2. MLP로 MNIST 분류하기

#### 2.1 데이터 가져오기

In [14]:
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, cache=True)
mnist.target = mnist.target.astype(np.int8)

X = mnist.data / 256
Y = mnist.target



# 3. Overfitting 방지하기

## 3.1 모델 복잡도 줄이기
## 3.2 Regularization 추가하기
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6, weight_decay=1e-5)
## 3.3 Dropout
Training 과정에서 일부러 몇 개 뉴런을 사용하지 않는 것. <br />
효과 1. 특정 조합에 너무 의존적이게 되는 것을 방지<br />
효과 2. 랜덤하게 dropout하면, 서로 다른 여러 신경망의 앙상블을 사용하는 것과 같은 효과가 있다

